In [1]:
import pandas as pd

In [4]:
result = pd.read_csv('/proceed/train_data_all.csv',encoding='utf-8',sep='|')

In [5]:
result.head()

,id,ret
0,25001,用户来电对01月产生的15.93元的流量费有疑义，我方经系统查询安上网日志向用户解释，用户不...
1,25002,* * * * * * * * 客户对202011-202012月产生的11.12元的流量费...
2,25003,用户来电反映套餐到期没有通知，我方经系统查询告知用户套餐7 折合约到期在2 月底到期，用户强...
3,25004,用户来电反映自己在网厅办理的49元20g的5g升级包，用户称自己没有办理过，已向用户正常解释...
4,25005,用户来电反映自己办理的承诺通信费60元档得12gb 流量与权益（立即生效合约12个月）业务，...


In [11]:
import rouge
def compute_metrics(rg,preds, targets):
    """
    计算rouge指标
    :param rg:实例化的rouge对象
    :param preds: 文本1
    :param targets: 文本2
    :return: 输出为rouge1,rouge2,rougel的加权均值
    """
    score = rg.get_scores(' '.join(preds), ' '.join(targets))
    rouge1 = score[0]['rouge-1']['f']
    rouge2 = score[0]['rouge-2']['f']
    rougel = score[0]['rouge-l']['f']
    # print(round(rouge1*100, 2),round(rouge2*100, 2),round(rougel*100, 2))
    score = round((rouge1 * 0.2 + rouge2 * 0.4 + rougel * 0.4) * 100, 5)
    return score

In [12]:
text1 = "今天天气不错"
text2 = "今天天气不好"
compute_metrics(rouge.Rouge(),text1,text2)

0.8